# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import csv

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#load csv
city_df=pd.read_csv("output_data/cities.csv", encoding="utf-8")
city_df.head()


,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
0,Arrecife,28.9630,-13.5477,71.67,64,0,9.22,ES,1643985399
1,Dikson,73.5069,80.5464,-2.13,90,100,5.17,RU,1643985125
2,Lüderitz,-26.6481,15.1594,76.68,58,4,15.84,NaN,1643985399
3,Rawson,-43.3002,-65.1023,63.86,70,98,22.86,AR,1643985399
4,Barrow,71.2906,-156.7887,-29.18,75,0,5.75,US,1643985400


In [3]:
#create a coordinate column that combines lat & lon 
city_df['coord']= list(zip(city_df['Latitude'], city_df['Longitude']))
city_df.head()

,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date,coord
0,Arrecife,28.9630,-13.5477,71.67,64,0,9.22,ES,1643985399,"(28.963, -13.5477)"
1,Dikson,73.5069,80.5464,-2.13,90,100,5.17,RU,1643985125,"(73.5069, 80.5464)"
2,Lüderitz,-26.6481,15.1594,76.68,58,4,15.84,NaN,1643985399,"(-26.6481, 15.1594)"
3,Rawson,-43.3002,-65.1023,63.86,70,98,22.86,AR,1643985399,"(-43.3002, -65.1023)"
4,Barrow,71.2906,-156.7887,-29.18,75,0,5.75,US,1643985400,"(71.2906, -156.7887)"


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [5]:
figure_layout = {
    'width': '800px',
    'height': '500px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}


In [6]:
#set variables for plotting
locations = city_df['coord']
humid = city_df['Humidity']
maxtemp= city_df['Max Temperature'].max()
maxtemps=maxtemp

# Plot Heatmap
fig = gmaps.figure(layout=figure_layout, zoom_level=2,center=(20,40))

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humid, 
                                 dissipating=False, max_intensity=maxtemps,
                                 point_radius=2)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
# My ideal location (for purposes of this exercise): not humid, not hot, pretty cloudy
vacation_df=city_df.loc[(city_df['Humidity'] <= 42) & (city_df['Max Temperature'] <= 80)& 
                        (city_df['Cloudiness'] > 70),:]
vacation_df

,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date,coord
183,Kumul,42.8000,93.4500,25.29,34,84,4.41,CN,1643985547,"(42.8, 93.45)"
261,Tiznit Province,29.5833,-9.5000,74.41,21,84,8.68,MA,1643985631,"(29.5833, -9.5)"
289,Baghdad,33.3406,44.4009,62.51,42,75,18.41,IQ,1643985639,"(33.3406, 44.4009)"
301,Orzu,37.5593,68.8184,48.60,32,100,7.76,TJ,1643985672,"(37.5593, 68.8184)"
302,Ondjiva,-17.0667,15.7333,77.85,37,100,5.23,AO,1643985672,"(-17.0667, 15.7333)"
314,Zhangye,38.9342,100.4517,20.62,32,100,3.40,CN,1643985548,"(38.9342, 100.4517)"
403,Yumen,40.2833,97.2000,18.19,40,100,13.65,CN,1643985766,"(40.2833, 97.2)"
405,Mīzan Teferī,6.9833,35.5833,77.20,40,99,4.12,ET,1643985766,"(6.9833, 35.5833)"
406,Mbarara,-0.6080,30.6500,77.52,42,100,6.51,UG,1643985767,"(-0.608, 30.65)"
466,Yovon,38.3141,69.0378,45.81,33,99,7.29,TJ,1643985819,"(38.3141, 69.0378)"


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
# set up additional column to hold hotel name
vacation_df["hotel"] = ""

/var/folders/qc/_ykx45d102d0nr51ysx4tb4h0000gn/T/ipykernel_1435/2656888937.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vacation_df["hotel"] = ""


In [9]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "keyword": "Hotel",
    "key": g_key
    }

for index, row in vacation_df.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    #lat and lon inside the loop so it only takes for that row and add it to search params  
    lat=row["Latitude"]
    lon=row["Longitude"]
    params["location"]= f"{lat},{lon}"
    
    # make request and collect the json into list
    hotel_list = requests.get(base_url, params=params).json()
    
    
    #extract hotel name from json in appropriate place and add it to the dataframe
    #add try/except to catch locations with no hotels within range
    #add message to the df if no results found so there are not blanks
    try: 
        vacation_df.loc[index, 'hotel'] = hotel_list["results"][0]["name"]
    except: 
        vacation_df.loc[index, 'hotel'] = "No Nearby Hotels"
       

/Users/michaelraines/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [10]:
#confirm loop & try/except worked
vacation_df

,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date,coord,hotel
183,Kumul,42.8000,93.4500,25.29,34,84,4.41,CN,1643985547,"(42.8, 93.45)",Π Hotel
261,Tiznit Province,29.5833,-9.5000,74.41,21,84,8.68,MA,1643985631,"(29.5833, -9.5)",No Nearby Hotels
289,Baghdad,33.3406,44.4009,62.51,42,75,18.41,IQ,1643985639,"(33.3406, 44.4009)",Shanashel Palace Hotel
301,Orzu,37.5593,68.8184,48.60,32,100,7.76,TJ,1643985672,"(37.5593, 68.8184)",No Nearby Hotels
302,Ondjiva,-17.0667,15.7333,77.85,37,100,5.23,AO,1643985672,"(-17.0667, 15.7333)",Águia Verde Hotel
314,Zhangye,38.9342,100.4517,20.62,32,100,3.40,CN,1643985548,"(38.9342, 100.4517)",Huachen International Hotel Zhangye
403,Yumen,40.2833,97.2000,18.19,40,100,13.65,CN,1643985766,"(40.2833, 97.2)",No Nearby Hotels
405,Mīzan Teferī,6.9833,35.5833,77.20,40,99,4.12,ET,1643985766,"(6.9833, 35.5833)",Kashinin Hotel Ethiopia
406,Mbarara,-0.6080,30.6500,77.52,42,100,6.51,UG,1643985767,"(-0.608, 30.65)",Lake View Resort Hotel
466,Yovon,38.3141,69.0378,45.81,33,99,7.29,TJ,1643985819,"(38.3141, 69.0378)",No Nearby Hotels


In [11]:
#change necessary variables to match up with what I've called things 

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in vacation_df.iterrows()]
locations = vacation_df[["Latitude", "Longitude"]]



In [12]:
# Add marker layer ontop of heat map using the variables established above
markers = gmaps.marker_layer(locations,info_box_content=hotel_info)
#add layer to existing
fig.add_layer(markers)
#display
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…